In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import ConcatDataset
from torchvision import transforms
from PIL import Image
import numpy as np
import torchvision.models as models


In [ ]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.backends.mps.is_available())

if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple GPU
elif torch.cuda.is_available():
    device = torch.device("cuda") # NVIDIA GPU
else:
    device = torch.device("cpu")  #fallback
print("Using device:", device)



cuda
False
Using device: cuda


In [37]:
df = pd.read_csv('fashion-mnist_train.csv')
#df = pd.read_csv('fmnist_small.csv')
df.head()
df.shape


(60000, 785)

In [38]:
#train test split data
x = df.iloc[:,1:].values #pixel values starting from col 1
y = df.iloc[:,0].values # y is the label


In [39]:
x_train_full , x_test, y_train_full, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
print("x_train_full shape:", x_train_full.shape)
print("y_train_full shape:", y_train_full.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)



x_train_full shape: (48000, 784)
y_train_full shape: (48000,)
x_test shape: (12000, 784)
y_test shape: (12000,)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full, test_size=0.1, random_state=42
)


In [41]:
#transformations

custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
#create customdataset class
class CustomDataset(Dataset):

    def __init__(self, features, labels,transform):
        #convert to pytroch tensors
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28) ## change -1
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        #resize to 28x28
        image = self.features[index].reshape(28,28)

        #change data type to np.uint8
        #image = image.astype(np.uint8)
        image = image.numpy().astype(np.uint8)

        #change to color & (H,W,C) to C,H,W
        image = np.stack([image]*3 , axis=-1)
        #convert array to PIL img
        image = Image.fromarray(image)

        #apply custom transforms
        image = self.transform(image)
        #if self.transform:
           # image = self.transform(image)

        #return
        return image, torch.tensor(self.labels[index], dtype= torch.long) 


In [43]:
# create train dataset object
train_dataset = CustomDataset(x_train,y_train, transform=custom_transform)
#create test dataset object
test_dataset = CustomDataset(x_test,y_test, transform=custom_transform)
val_dataset = CustomDataset(x_val,y_val,transform=custom_transform)




In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

#val loader

In [45]:
#get vgg16
vgg16 = models.vgg16(pretrained = True)

#freeze conv / features layers
for param in vgg16.features.parameters():
    param.requires_grad = False


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [46]:
#our own classifier
vgg16.classifier = nn.Sequential(
    nn.Linear(25088,1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512,10)
)
vgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [47]:
learning_rate = 0.0001
epochs = 6
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [48]:
#training loop

for epoch in range(epochs):
    tota_epoch_loss = 0

    for batch_featueres, batch_labels in train_loader:
        batch_featueres , batch_labels = batch_featueres.to(device) , batch_labels.to(device)

        outputs = vgg16(batch_featueres)
        #print(outputs.shape)
        #print(batch_labels.shape)

        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tota_epoch_loss = tota_epoch_loss + loss.item()

    avg_loss = tota_epoch_loss / len(train_loader)
    print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

/tmp/ipython-input-4073485404.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return image, torch.tensor(self.labels[index], dtype= torch.long)  #why are we doing this tensor


Epoch: 1 , Loss: 0.3583094042594786
Epoch: 2 , Loss: 0.21347526237368583
Epoch: 3 , Loss: 0.16314352163769028
Epoch: 4 , Loss: 0.12834874063316318
Epoch: 5 , Loss: 0.09951428454826344
Epoch: 6 , Loss: 0.07855323327843237


In [ ]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [50]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

/tmp/ipython-input-4073485404.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return image, torch.tensor(self.labels[index], dtype= torch.long)  #why are we doing this tensor


0.927


In [51]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

/tmp/ipython-input-4073485404.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return image, torch.tensor(self.labels[index], dtype= torch.long)  #why are we doing this tensor


0.991550925925926


In [1]:
#save model
torch.save(vgg16.state_dict(), "vgg16_fmnist.pth")

NameError: name 'torch' is not defined

In [4]:
# #results:
# Epoch: 1 , Loss: 0.3583094042594786
# Epoch: 2 , Loss: 0.21347526237368583
# Epoch: 3 , Loss: 0.16314352163769028
# Epoch: 4 , Loss: 0.12834874063316318
# Epoch: 5 , Loss: 0.09951428454826344
# Epoch: 6 , Loss: 0.07855323327843237

# test data: 0.927
# training data: 0.9915509